In [ ]:
%matplotlib inline


# 1D Rectilinear Mesh


In [ ]:
from geobipy import StatArray
from geobipy import RectilinearMesh1D
import matplotlib.pyplot as plt
import numpy as np

Instantiate a new 1D rectilinear mesh by specifying cell centres or edges.
Here we use edges



In [ ]:
x = StatArray(np.cumsum(np.arange(0.0, 10.0)), 'Depth', 'm')

In [ ]:
rm = RectilinearMesh1D(cellEdges=x)

In [ ]:
print(rm.cellCentres)

In [ ]:
print(rm.cellEdges)

In [ ]:
print(rm.internalCellEdges)

In [ ]:
print(rm.cellWidths)

Get the cell indices



In [ ]:
print(rm.cellIndex(np.r_[1.0, 5.0, 20.0]))

We can plot the grid of the mesh



In [ ]:
plt.figure()
_ = rm.plotGrid(flipY=True)

Or Pcolor the mesh showing. An array of cell values is used as the colour.



In [ ]:
plt.figure()
arr = StatArray(np.random.randn(rm.nCells), "Name", "Units")
_ = rm.pcolor(arr, grid=True, flipY=True)

Instantiate a new 1D rectilinear mesh by specifying cell centres or edges.
Here we use edges



In [ ]:
x = StatArray(np.logspace(-3, 3, 10), 'Depth', 'm')

In [ ]:
rm = RectilinearMesh1D(cellEdges=x, log=10)

Access property describing the mesh



In [ ]:
print(rm.cellCentres)

In [ ]:
print(rm.cellEdges)

In [ ]:
print(rm.internalCellEdges)

In [ ]:
print(rm.cellWidths)

Get the cell indices



In [ ]:
print(rm.cellIndex(np.r_[0.03, 5.0, 200.0]))

We can plot the grid of the mesh



In [ ]:
plt.figure()
_ = rm.plotGrid(flipY=True)

Or Pcolor the mesh showing. An array of cell values is used as the colour.



In [ ]:
plt.figure()
arr = StatArray(np.random.randn(rm.nCells), "Name", "Units")
_ = rm.pcolor(arr, grid=True, flipY=True)